In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_feather("dataFrame.feather")
data["category"] = pd.Categorical(data["category"])
print(data.shape)
X = data["text"]
Y = pd.get_dummies(data["category"])
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.2)

(2225, 3)


In [22]:
# that would be great but you dont have enough memory
import gensim

# Load Google's pre-trained Word2Vec model.
#model = gensim.models.KeyedVectors.load_word2vec_format('/home/robertb/Desktop/text_embeddings/project/GoogleNews-vectors-negative300.bin', binary=True)  
model_w2v = gensim.models.Word2Vec.load('/home/robertb/Desktop/text_embeddings/project/simple_word2vec.model')  


In [2]:
# # learn embedings
# import gensim 
# from nltk.tokenize import sent_tokenize, word_tokenize

# text_new_line = [data.iloc[i,1].replace('\n', ". ") for i in range(data.shape[0])]
# text_tokenized = [word_tokenize(text_new_line[i]) for i in range(len(text_new_line))]

# model = gensim.models.Word2Vec(
#         text_tokenized,
#         size=100,
#         window=6,
#         min_count=1,
#         workers=4,
#         iter=10)

# model.save("simple_word2vec.model")


In [43]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# get pairs word:index(word) 
vocabulary = {token: token_index for token_index, token in enumerate(model_w2v.wv.index2word)} 
# change words to integer seq using indexes
tk = Tokenizer(num_words=len(vocabulary)) 
tk.word_index = vocabulary 

train_text_seq = tk.texts_to_sequences(train_X)
maxlen = max([len(text) for text in train_text_seq])
train_text_seq = pad_sequences(train_text_seq, maxlen=maxlen, padding="post", value=0.0)

test_text_seq = tk.texts_to_sequences(test_X)
test_text_seq = pad_sequences(test_text_seq, maxlen=maxlen, padding="post", value=0.0)


# get word embedding matrix
#embedding_matrix = model_w2v.wv.get_keras_embedding(train_embeddings=False)


In [47]:
import keras.models as km
from keras.layers import Activation, Dense, Flatten, GlobalAveragePooling1D, Embedding
# define the model
model_NN = km.Sequential()
model_NN.add(Embedding(len(vocabulary)+1,100,trainable=True))
model_NN.add(GlobalAveragePooling1D())
model_NN.add(Dense(5, activation='sigmoid'))
# compile the model
model_NN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model_NN.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         4226800   
_________________________________________________________________
global_average_pooling1d_2 ( (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 505       
Total params: 4,227,305
Trainable params: 4,227,305
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
history=model_NN.fit(train_text_seq,
                  train_Y,
                  epochs=10,
                  batch_size=128,
                  validation_data=(test_text_seq, test_Y),
                  verbose=1)

/home/robertb/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1780 samples, validate on 445 samples
Epoch 1/10
1780/1780 [==============================] - 5s 3ms/step - loss: 0.6796 - accuracy: 0.6554 - val_loss: 0.6620 - val_accuracy: 0.8000
Epoch 2/10
1780/1780 [==============================] - 5s 3ms/step - loss: 0.6470 - accuracy: 0.8000 - val_loss: 0.6296 - val_accuracy: 0.8000
Epoch 3/10
1780/1780 [==============================] - 5s 3ms/step - loss: 0.6143 - accuracy: 0.8000 - val_loss: 0.5962 - val_accuracy: 0.8000
Epoch 4/10
1780/1780 [==============================] - 5s 3ms/step - loss: 0.5805 - accuracy: 0.8000 - val_loss: 0.5628 - val_accuracy: 0.8000
Epoch 5/10
1780/1780 [==============================] - 5s 3ms/step - loss: 0.5487 - accuracy: 0.8000 - val_loss: 0.5336 - val_accuracy: 0.8000
Epoch 6/10
1780/1780 [==============================] - 5s 3ms/step - loss: 0.5233 - accuracy: 0.8000 - val_loss: 0.5133 - val_accuracy: 0.8000
Epoch 7/10
1780/1780 [==============================] - 5s 3ms/step - loss: 0.5078 - accu

In [70]:
data.shape[0]

2225